# 使用静态图加速

## 背景介绍

AI编译框架分为两种运行模式，分别是动态图模式以及静态图模式。MindSpore默认情况下是以动态图模式运行，但也支持手工切换为静态图模式。两种运行模式的详细介绍如下：

### 动态图模式

动态图的特点是计算图的构建和计算同时发生（Define by run），其符合Python的解释执行方式，在计算图中定义一个Tensor时，其值就已经被计算且确定，因此在调试模型时较为方便，能够实时得到中间结果的值，但由于所有节点都需要被保存，导致难以对整个计算图进行优化。

在MindSpore中，动态图模式又被称为PyNative模式。由于动态图的解释执行特性，在脚本开发和网络流程调试过程中，推荐使用动态图模式进行调试。
如需要手动控制框架采用PyNative模式，可以通过以下代码进行网络构建：

In [1]:
%%capture captured_output
# 实验环境已经预装了mindspore==2.4.10，如需更换mindspore版本，可更改下面 MINDSPORE_VERSION 变量
!pip uninstall mindspore -y
%env MINDSPORE_VERSION=2.4.10
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/${MINDSPORE_VERSION}/MindSpore/unified/aarch64/mindspore-${MINDSPORE_VERSION}-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

In [2]:
# 查看当前 mindspore 版本
!pip show mindspore

Name: mindspore
Version: 2.4.10
Summary: MindSpore is a new open source deep learning training/inference framework that could be used for mobile, edge and cloud scenarios.
Home-page: https://www.mindspore.cn
Author: The MindSpore Authors
Author-email: contact@mindspore.cn
License: Apache 2.0
Location: /home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages
Requires: asttokens, astunparse, numpy, packaging, pillow, protobuf, psutil, safetensors, scipy
Required-by: 


In [3]:
import numpy as np
import mindspore as ms
from mindspore import nn, Tensor
ms.set_context(mode=ms.PYNATIVE_MODE)  # 使用set_context进行动态图模式的配置

class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()
input = Tensor(np.ones([64, 1, 28, 28]).astype(np.float32))
output = model(input)
print(output)

/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


[[-0.05153345  0.15412588 -0.04261633  0.06199193 -0.13109688  0.17259671
  -0.10635071  0.01608355 -0.02379631  0.15235695]
 [-0.05153345  0.15412588 -0.04261633  0.06199193 -0.13109688  0.17259671
  -0.10635071  0.01608355 -0.02379631  0.15235695]
 [-0.05153345  0.15412588 -0.04261633  0.06199193 -0.13109688  0.17259671
  -0.10635071  0.01608355 -0.02379631  0.15235695]
 [-0.05153345  0.15412588 -0.04261633  0.06199193 -0.13109688  0.17259671
  -0.10635071  0.01608355 -0.02379631  0.15235695]
 [-0.05153345  0.15412588 -0.04261633  0.06199193 -0.13109688  0.17259671
  -0.10635071  0.01608355 -0.02379631  0.15235695]
 [-0.05153345  0.15412588 -0.04261633  0.06199193 -0.13109688  0.17259671
  -0.10635071  0.01608355 -0.02379631  0.15235695]
 [-0.05153345  0.15412588 -0.04261633  0.06199193 -0.13109688  0.17259671
  -0.10635071  0.01608355 -0.02379631  0.15235695]
 [-0.05153345  0.15412588 -0.04261633  0.06199193 -0.13109688  0.17259671
  -0.10635071  0.01608355 -0.02379631  0.15235695]


### 静态图模式

相较于动态图而言，静态图的特点是将计算图的构建和实际计算分开（Define and run）。有关静态图模式的运行原理，可以参考[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/r2.4.10/model_train/program_form/static_graph.html#概述)。

在MindSpore中，静态图模式又被称为Graph模式，在Graph模式下，基于图优化、计算图整图下沉等技术，编译器可以针对图进行全局的优化，获得较好的性能，因此比较适合网络固定且需要高性能的场景。

如需要手动控制框架采用静态图模式，可以通过以下代码进行网络构建：

In [4]:
import numpy as np
import mindspore as ms
from mindspore import nn, Tensor
ms.set_context(mode=ms.GRAPH_MODE)  # 使用set_context进行运行静态图模式的配置

class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()
input = Tensor(np.ones([64, 1, 28, 28]).astype(np.float32))
output = model(input)
print(output)

.[[-0.01968384  0.01934814  0.20678711 -0.21679688 -0.11401367  0.12805176
   0.00195026  0.0061264   0.02903748  0.0355835 ]
 [-0.01968384  0.01934814  0.20678711 -0.21679688 -0.11401367  0.12805176
   0.00195026  0.0061264   0.02903748  0.0355835 ]
 [-0.01968384  0.01934814  0.20678711 -0.21679688 -0.11401367  0.12805176
   0.00195026  0.0061264   0.02903748  0.0355835 ]
 [-0.01968384  0.01934814  0.20678711 -0.21679688 -0.11401367  0.12805176
   0.00195026  0.0061264   0.02903748  0.0355835 ]
 [-0.01968384  0.01934814  0.20678711 -0.21679688 -0.11401367  0.12805176
   0.00195026  0.0061264   0.02903748  0.0355835 ]
 [-0.01968384  0.01934814  0.20678711 -0.21679688 -0.11401367  0.12805176
   0.00195026  0.0061264   0.02903748  0.0355835 ]
 [-0.01968384  0.01934814  0.20678711 -0.21679688 -0.11401367  0.12805176
   0.00195026  0.0061264   0.02903748  0.0355835 ]
 [-0.01968384  0.01934814  0.20678711 -0.21679688 -0.11401367  0.12805176
   0.00195026  0.0061264   0.02903748  0.0355835 ]

## 静态图模式的使用场景

MindSpore编译器重点面向Tensor数据的计算以及其微分处理。因此使用MindSpore API以及基于Tensor对象的操作更适合使用静态图编译优化。其他操作虽然可以部分入图编译，但实际优化作用有限。另外，静态图模式先编译后执行的模式导致其存在编译耗时。因此，如果函数无需反复执行，那么使用静态图加速也可能没有价值。

有关使用静态图来进行网络编译的示例，请参考[网络构建](https://www.mindspore.cn/tutorials/zh-CN/r2.4.10/beginner/model.html)。

## 静态图模式开启方式

通常情况下，由于动态图的灵活性，我们会选择使用PyNative模式来进行自由的神经网络构建，以实现模型的创新和优化。但是当需要进行性能加速时，我们需要对神经网络部分或整体进行加速。MindSpore提供了两种切换为图模式的方式，分别是基于装饰器的开启方式以及基于全局context的开启方式。

### 基于装饰器的开启方式

MindSpore提供了jit装饰器，可以通过修饰Python函数或者Python类的成员函数使其被编译成计算图，通过图优化等技术提高运行速度。此时我们可以简单的对想要进行性能优化的模块进行图编译加速，而模型其他部分，仍旧使用解释执行方式，不丢失动态图的灵活性。无论全局context是设置成静态图模式还是动态图模式，被jit修饰的部分始终会以静态图模式进行运行。

在需要对Tensor的某些运算进行编译加速时，可以在其定义的函数上使用jit修饰器，在调用该函数时，该模块自动被编译为静态图。需要注意的是，jit装饰器只能用来修饰函数，无法对类进行修饰。jit的使用示例如下：

In [5]:
import numpy as np
import mindspore as ms
from mindspore import nn, Tensor

class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

input = Tensor(np.ones([64, 1, 28, 28]).astype(np.float32))

@ms.jit  # 使用ms.jit装饰器，使被装饰的函数以静态图模式运行
def run(x):
    model = Network()
    return model(x)

output = run(input)
print(output)

[[ 0.0737915  -0.18676758 -0.04821777 -0.0289917  -0.08178711  0.17883301
  -0.15124512  0.06518555 -0.00880432  0.09594727]
 [ 0.0737915  -0.18676758 -0.04821777 -0.0289917  -0.08178711  0.17883301
  -0.15124512  0.06518555 -0.00880432  0.09594727]
 [ 0.0737915  -0.18676758 -0.04821777 -0.0289917  -0.08178711  0.17883301
  -0.15124512  0.06518555 -0.00880432  0.09594727]
 [ 0.0737915  -0.18676758 -0.04821777 -0.0289917  -0.08178711  0.17883301
  -0.15124512  0.06518555 -0.00880432  0.09594727]
 [ 0.0737915  -0.18676758 -0.04821777 -0.0289917  -0.08178711  0.17883301
  -0.15124512  0.06518555 -0.00880432  0.09594727]
 [ 0.0737915  -0.18676758 -0.04821777 -0.0289917  -0.08178711  0.17883301
  -0.15124512  0.06518555 -0.00880432  0.09594727]
 [ 0.0737915  -0.18676758 -0.04821777 -0.0289917  -0.08178711  0.17883301
  -0.15124512  0.06518555 -0.00880432  0.09594727]
 [ 0.0737915  -0.18676758 -0.04821777 -0.0289917  -0.08178711  0.17883301
  -0.15124512  0.06518555 -0.00880432  0.09594727]


除使用修饰器外，也可使用函数变换方式调用jit方法，示例如下：

In [6]:
import numpy as np
import mindspore as ms
from mindspore import nn, Tensor

class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

input = Tensor(np.ones([64, 1, 28, 28]).astype(np.float32))

def run(x):
    model = Network()
    return model(x)

run_with_jit = ms.jit(run)  # 通过调用jit将函数转换为以静态图方式执行
output = run_with_jit(input)
print(output)

[[-0.18347168  0.04364014  0.13378906 -0.3178711   0.04345703  0.02009583
  -0.10748291  0.03869629 -0.06063843 -0.03659058]
 [-0.18347168  0.04364014  0.13378906 -0.3178711   0.04345703  0.02009583
  -0.10748291  0.03869629 -0.06063843 -0.03659058]
 [-0.18347168  0.04364014  0.13378906 -0.3178711   0.04345703  0.02009583
  -0.10748291  0.03869629 -0.06063843 -0.03659058]
 [-0.18347168  0.04364014  0.13378906 -0.3178711   0.04345703  0.02009583
  -0.10748291  0.03869629 -0.06063843 -0.03659058]
 [-0.18347168  0.04364014  0.13378906 -0.3178711   0.04345703  0.02009583
  -0.10748291  0.03869629 -0.06063843 -0.03659058]
 [-0.18347168  0.04364014  0.13378906 -0.3178711   0.04345703  0.02009583
  -0.10748291  0.03869629 -0.06063843 -0.03659058]
 [-0.18347168  0.04364014  0.13378906 -0.3178711   0.04345703  0.02009583
  -0.10748291  0.03869629 -0.06063843 -0.03659058]
 [-0.18347168  0.04364014  0.13378906 -0.3178711   0.04345703  0.02009583
  -0.10748291  0.03869629 -0.06063843 -0.03659058]


当我们需要对神经网络的某部分进行加速时，可以直接在construct方法上使用jit修饰器，在调用实例化对象时，该模块自动被编译为静态图。示例如下：

In [7]:
import numpy as np
import mindspore as ms
from mindspore import nn, Tensor

class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    @ms.jit  # 使用ms.jit装饰器，使被装饰的函数以静态图模式运行
    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

input = Tensor(np.ones([64, 1, 28, 28]).astype(np.float32))
model = Network()
output = model(input)
print(output)

[[ 0.04376221  0.09332275  0.01655579  0.02799988 -0.01991272  0.06671143
  -0.01029205 -0.0296936   0.03485107 -0.04522705]
 [ 0.04376221  0.09332275  0.01655579  0.02799988 -0.01991272  0.06671143
  -0.01029205 -0.0296936   0.03485107 -0.04522705]
 [ 0.04376221  0.09332275  0.01655579  0.02799988 -0.01991272  0.06671143
  -0.01029205 -0.0296936   0.03485107 -0.04522705]
 [ 0.04376221  0.09332275  0.01655579  0.02799988 -0.01991272  0.06671143
  -0.01029205 -0.0296936   0.03485107 -0.04522705]
 [ 0.04376221  0.09332275  0.01655579  0.02799988 -0.01991272  0.06671143
  -0.01029205 -0.0296936   0.03485107 -0.04522705]
 [ 0.04376221  0.09332275  0.01655579  0.02799988 -0.01991272  0.06671143
  -0.01029205 -0.0296936   0.03485107 -0.04522705]
 [ 0.04376221  0.09332275  0.01655579  0.02799988 -0.01991272  0.06671143
  -0.01029205 -0.0296936   0.03485107 -0.04522705]
 [ 0.04376221  0.09332275  0.01655579  0.02799988 -0.01991272  0.06671143
  -0.01029205 -0.0296936   0.03485107 -0.04522705]


### 基于context的开启方式

context模式是一种全局的设置模式。代码示例如下：

In [8]:
import numpy as np
import mindspore as ms
from mindspore import nn, Tensor
ms.set_context(mode=ms.GRAPH_MODE)  # 使用set_context进行运行静态图模式的配置

class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()
input = Tensor(np.ones([64, 1, 28, 28]).astype(np.float32))
output = model(input)
print(output)

[[ 0.00716019  0.12060547 -0.07086182 -0.02961731  0.06695557  0.04946899
   0.02612305  0.2088623   0.02856445 -0.06817627]
 [ 0.00716019  0.12060547 -0.07086182 -0.02961731  0.06695557  0.04946899
   0.02612305  0.2088623   0.02856445 -0.06817627]
 [ 0.00716019  0.12060547 -0.07086182 -0.02961731  0.06695557  0.04946899
   0.02612305  0.2088623   0.02856445 -0.06817627]
 [ 0.00716019  0.12060547 -0.07086182 -0.02961731  0.06695557  0.04946899
   0.02612305  0.2088623   0.02856445 -0.06817627]
 [ 0.00716019  0.12060547 -0.07086182 -0.02961731  0.06695557  0.04946899
   0.02612305  0.2088623   0.02856445 -0.06817627]
 [ 0.00716019  0.12060547 -0.07086182 -0.02961731  0.06695557  0.04946899
   0.02612305  0.2088623   0.02856445 -0.06817627]
 [ 0.00716019  0.12060547 -0.07086182 -0.02961731  0.06695557  0.04946899
   0.02612305  0.2088623   0.02856445 -0.06817627]
 [ 0.00716019  0.12060547 -0.07086182 -0.02961731  0.06695557  0.04946899
   0.02612305  0.2088623   0.02856445 -0.06817627]


## 静态图的语法约束

在Graph模式下，Python代码并不是由Python解释器去执行，而是将代码编译成静态计算图，然后执行静态计算图。因此，编译器无法支持全量的Python语法。MindSpore的静态图编译器维护了Python常用语法子集，以支持神经网络的构建及训练。详情可参考[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/r2.4.10/model_train/program_form/static_graph.html)。

## 静态图高级编程技巧

使用静态图高级编程技巧可以有效地提高编译效率以及执行效率，并可以使程序运行的更加稳定。详情可参考[静态图高级编程技巧](https://www.mindspore.cn/docs/zh-CN/r2.4.10/model_train/program_form/static_graph_syntax/static_graph_expert_programming.html)。